In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
%run "/eReport/PkgRcd/Cálculo de Provisiones"

In [0]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

class PRC_RCD_ALINEAMIENTO_INTERES():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.empresa = parametros['empresa']  
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d') 
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
      
  def ProcesaAlineamientoInteres(self):
    try:
      #Reproceso
      limpiaAlineamiento = """UPDATE B
      SET B.ALINEACION = NULL
      FROM CLIENTE A JOIN CLIENTE_PRODUCTO B
      ON A.ID = B.CLIENTE
      WHERE A.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""';
      """
      self.cursor.execute(limpiaAlineamiento)
      self.connection.commit()

      #Reproceso
      clientesQueAlinearon = """SELECT D.ID AS ID_CPC,
                                 CD.EMPRESA,
                                 C.PRODUCTO,
                                 B.TIPO_CLIENTE,
                                 B.TIPO_CREDITO,
                                 B.TIPO_CREDITO_SBS,
                                 B.SITUACION_CREDITO,
                                 B.INDICADOR_PAGO_CASTIGO,
                                 A.CLASIFICACION_DEUDOR,
                                 D.CONCEPTO_SALDO_PRINCIPAL,
                                 D.CONCEPTO_SALDO,
                                 D.CONCEPTO_CLASIFICACION,
                                 D.MONEDA,
                                 D.CUENTA_CONTABLE
                          FROM CLIENTE A
                          JOIN CLIENTE_DEUDOR CD ON CD.ID = A.CLIENTE_DEUDOR
                          JOIN CLIENTE_PRODUCTO B ON A.ID = B.CLIENTE
                          JOIN CLIENTE_DEUDOR_PRODUCTO C ON C.ID = B.CLIENTE_DEUDOR_PRODUCTO
                          JOIN CLIENTE_PRODUCTO_CONCEPTO D ON B.ID = D.CLIENTE_PRODUCTO
                          JOIN PROCESO_PRODUCTO E ON E.PROCESO = 2
                          AND E.PRODUCTO = C.PRODUCTO
                          JOIN PROCESO_PRODUCTO_DETALLE F ON E.ID = F.PROCESO_PRODUCTO
                          AND F.GRUPO_CONCEPTO_SALDO = 2
                          AND F.CONCEPTO_SALDO = D.CONCEPTO_SALDO
                          AND F.SITUACION_CREDITO = B.SITUACION_CREDITO
                          WHERE A.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
                            AND D.CONCEPTO_CLASIFICACION IN (3, 6);"""

      sql_clientesQueAlinearon = pd.read_sql_query(clientesQueAlinearon, self.connection)
      df_clientesQueAlinearon = pd.DataFrame(sql_clientesQueAlinearon)

      if not df_clientesQueAlinearon.empty:
        #relacionamos con cuenta contable
        df_clientesQueAlinearon = pd.merge(df_clientesQueAlinearon, PRC_RCD_PROVISIONES(1).cuentaContableVigente(self.fecha_proceso_str), on=['EMPRESA','PRODUCTO','TIPO_CLIENTE','TIPO_CREDITO','TIPO_CREDITO_SBS','SITUACION_CREDITO','INDICADOR_PAGO_CASTIGO','CLASIFICACION_DEUDOR','CONCEPTO_SALDO_PRINCIPAL','CONCEPTO_SALDO','MONEDA','CONCEPTO_CLASIFICACION'], how='left')

        df_clientesQueAlinearon['CONCEPTO_CLASIFICACION_y'] = np.where(df_clientesQueAlinearon['CONCEPTO_CLASIFICACION_x'] == 3, 
                                                                       1, np.where(df_clientesQueAlinearon['CONCEPTO_CLASIFICACION_x'] == 6, 4))
        df_clientesQueAlinearon['CONCEPTO_CLASIFICACION_z'] = np.where(df_clientesQueAlinearon['CONCEPTO_CLASIFICACION_x'] == 3, 
                                                                       1, np.where(df_clientesQueAlinearon['CONCEPTO_CLASIFICACION_x'] == 6, 4))

        df_clientesQueAlinearon = df_clientesQueAlinearon[['CONCEPTO_CLASIFICACION_z','ID','ID_CPC']]
        #actualizamos el concepto clasificacion inicial
        reprocesaAlineacion = "UPDATE [DBO].CLIENTE_PRODUCTO_CONCEPTO SET CONCEPTO_CLASIFICACION= ?, CUENTA_CONTABLE=?, WHERE ID =?"
        self.cursor.fast_executemany = True
        self.cursor.executemany(reprocesaAlineacion, df_clientesQueAlinearon.values.tolist())
        self.connection.commit()

      #Proceso
      AlineamientoInteres = """SELECT B.ID AS CLIENTE_PRODUCTO, D.ID AS CLIENTE_PRODUCTO_CONCEPTO,
               CD.EMPRESA,
               C.PRODUCTO,
               B.TIPO_CLIENTE,
               B.TIPO_CREDITO,
               B.TIPO_CREDITO_SBS,
               B.SITUACION_CREDITO,
               B.INDICADOR_PAGO_CASTIGO,
               A.CLASIFICACION_DEUDOR,
               D.CONCEPTO_SALDO_PRINCIPAL,
               D.CONCEPTO_SALDO,
               D.CONCEPTO_CLASIFICACION,
               D.MONEDA,
               D.CUENTA_CONTABLE
        FROM CLIENTE A
        JOIN CLIENTE_DEUDOR CD ON CD.ID = A.CLIENTE_DEUDOR
        JOIN CLIENTE_PRODUCTO B ON A.ID = B.CLIENTE
        JOIN CLIENTE_DEUDOR_PRODUCTO C ON C.ID = B.CLIENTE_DEUDOR_PRODUCTO
        JOIN CLIENTE_PRODUCTO_CONCEPTO D ON B.ID = D.CLIENTE_PRODUCTO
        JOIN PROCESO_PRODUCTO E ON E.PROCESO = 2
        AND E.PRODUCTO = C.PRODUCTO
        JOIN PROCESO_PRODUCTO_DETALLE F ON E.ID = F.PROCESO_PRODUCTO
        AND F.GRUPO_CONCEPTO_SALDO = 2
        AND F.CONCEPTO_SALDO = D.CONCEPTO_SALDO
        AND F.SITUACION_CREDITO = B.SITUACION_CREDITO
        WHERE A.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
          AND D.CONCEPTO_CLASIFICACION IN (1, 4)
          AND B.TIPO_CREDITO = '1';"""

      sql_AlineamientoInteres = pd.read_sql_query(AlineamientoInteres, self.connection)
      df_AlineamientoInteres = pd.DataFrame(sql_AlineamientoInteres)

      if not df_AlineamientoInteres.empty:

        #obtenemos la cuenta contable del alineamiento
        df_AlineamientoInteres = pd.merge(df_AlineamientoInteres, PRC_RCD_PROVISIONES(1).cuentaContableVigente(self.fecha_proceso_str),on=['EMPRESA','PRODUCTO','TIPO_CLIENTE','TIPO_CREDITO','TIPO_CREDITO_SBS','SITUACION_CREDITO','INDICADOR_PAGO_CASTIGO','CLASIFICACION_DEUDOR','CONCEPTO_SALDO_PRINCIPAL','CONCEPTO_SALDO','MONEDA','CONCEPTO_CLASIFICACION'], how='left')

        df_AlineamientoInteres['CONCEPTO_CLASIFICACION_y'] = np.where(df_AlineamientoInteres['CONCEPTO_CLASIFICACION_x'] == 1, 
                                                                      3, np.where(df_AlineamientoInteres['CONCEPTO_CLASIFICACION_x'] == 4, 6))
        df_AlineamientoInteres['CONCEPTO_CLASIFICACION_z'] = np.where(df_AlineamientoInteres['CONCEPTO_CLASIFICACION_x'] == 1, 
                                                                      3, np.where(df_AlineamientoInteres['CONCEPTO_CLASIFICACION_x'] == 4, 6))

        #nuevo dataframe con las operaciones que alinearon 
        df_operacionesAlineadas = df_AlineamientoInteres.loc[index,'CLIENTE_PRODUCTO'].dropDuplicates()

        #actualizamos concepto clasificacion y cuenta contable de alineamiento
        df_operacionesAlineadas = df_operacionesAlineadas[['CONCEPTO_CLASIFICACION_z','ID','CLIENTE_PRODUCTO_CONCEPTO']]

        #actualizamos el concepto clasificacion inicial
        actualizaAlineacion = "UPDATE [DBO].CLIENTE_PRODUCTO_CONCEPTO SET CONCEPTO_CLASIFICACION= ?, CUENTA_CONTABLE=? WHERE ID =?"
        self.cursor.fast_executemany = True
        self.cursor.executemany(actualizaAlineacion, df_operacionesAlineadas.values.tolist())
        self.connection.commit()

        df_operacionesAlineadas = df_operacionesAlineadas[['CLIENTE_PRODUCTO']]

        #activamos flag de alineamiento en las operaciones
        actualizaAlineacion = "UPDATE [DBO].CLIENTE_PRODUCTO SET ALINEACION='A' ? WHERE ID =?"
        self.cursor.fast_executemany = True
        self.cursor.executemany(actualizaAlineacion, df_operacionesAlineadas.values.tolist())
        self.connection.commit()
        print("proceso finalizado")
    except Exception as e:
      print("error: " + str(e))
      
#PRC_RCD_ALINEAMIENTO_INTERES = PRC_RCD_ALINEAMIENTO_INTERES(1)
#PRC_RCD_ALINEAMIENTO_INTERES.ProcesaAlineamientoInteres()      